In [ ]:
%matplotlib inline
import os
from helper_functions import *

db_file = os.path.abspath("db.json")

In [ ]:
from pymatgen.util.testing import PymatgenTest

struct_si = PymatgenTest.get_structure("Si")
print struct_si

In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet

vis = MPRelaxSet(struct_si, force_gamma=True)

In [ ]:
from matmethods.vasp.fireworks.core import OptimizeFW

fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=db_file)

for task in fw1.tasks:
    print task.fw_name

In [ ]:
from matmethods.vasp.fireworks.core import StaticFW

fw2 = StaticFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=db_file, 
               parents=[fw1])

for task in fw2.tasks:
    print task.fw_name

In [ ]:
from matmethods.vasp.fireworks.core import NonSCFFW

fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="uniform")

for task in fw3.tasks:
    print task.fw_name

In [ ]:
fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="line")

for task in fw4.tasks:
    print task.fw_name

In [ ]:
from fireworks import Workflow

fws = [fw1, fw2, fw3, fw4]

wf = Workflow(fws)

In [ ]:
plot_wf(wf)

In [ ]:
# adjust the ref_dir path
wf = simulate_bandstructure_vasprun(wf, ref_dir=os.path.abspath("."))

In [ ]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

In [ ]:
lp.add_wf(wf)